# Belongs to Airbnb Lab

### Introduction
In this lab we will continue to explore the relationships between data in different tables of a database. The Airbnb database for this lab contains four tables, `hosts`, `listings`, `locations`, and `neighborhoods`. In order to understand and analyze the data, we need to first understand the relationships between the tables. Relationships include "Has One" and "Has Many". For example, the `listings` table has a column "host_id" which HAS ONE record in hosts table that it corresponds to (a listing will only have one host). The `locations` table has an id column which HAS MANY corresponding records in the `listings` table (a location will have more than one listing).

Let's begin by connecting to the database and reviewing the schema of the tables.

### Loading Data

In [1]:
import pandas as pd
neighborhoods_url = "https://raw.githubusercontent.com/sql-fundamentals-jigsaw/mod-1-sql-curriculum/master/2-sql-relations/3-belongs-to-bnb/data/neighborhoods.csv"
hosts_url = "https://raw.githubusercontent.com/sql-fundamentals-jigsaw/mod-1-sql-curriculum/master/2-sql-relations/3-belongs-to-bnb/data/hosts.csv"
locations_url = "https://raw.githubusercontent.com/sql-fundamentals-jigsaw/mod-1-sql-curriculum/master/2-sql-relations/3-belongs-to-bnb/data/locations.csv"
listings_url = "https://raw.githubusercontent.com/sql-fundamentals-jigsaw/mod-1-sql-curriculum/master/2-sql-relations/3-belongs-to-bnb/data/listings.csv"


hosts_df = pd.read_csv(hosts_url)
neighborhoods_df = pd.read_csv(neighborhoods_url)

locations_df = pd.read_csv(locations_url)
listings_df = pd.read_csv(listings_url)

In [2]:
import sqlite3
conn = sqlite3.connect('listings.db')
cursor = conn.cursor()

In [3]:
hosts_df.to_sql('hosts',conn, index = False, if_exists = 'replace')
neighborhoods_df.to_sql('neighborhoods',conn, index = False, if_exists = 'replace')
locations_df.to_sql('locations',conn, index = False, if_exists = 'replace')
listings_df.to_sql('listings', conn, index = False, if_exists = 'replace')

48895

### Exploring Data

In [25]:
cursor.execute('SELECT name from sqlite_master where type= "table"')
cursor.fetchall()

[('hosts',), ('neighborhoods',), ('locations',), ('listings',)]

In [5]:
cursor.execute('PRAGMA table_info(hosts)')
cursor.fetchall()

[(0, 'id', 'INTEGER', 0, None, 0), (1, 'host_name', 'TEXT', 0, None, 0)]

In [6]:
cursor.execute('PRAGMA table_info(neighborhoods)')
cursor.fetchall()

[(0, 'id', 'INTEGER', 0, None, 0),
 (1, 'name', 'TEXT', 0, None, 0),
 (2, 'neighbourhood_group', 'TEXT', 0, None, 0)]

In [7]:
cursor.execute('PRAGMA table_info(locations)')
cursor.fetchall()

[(0, 'id', 'INTEGER', 0, None, 0),
 (1, 'longitude', 'REAL', 0, None, 0),
 (2, 'latitude', 'REAL', 0, None, 0),
 (3, 'neighborhood_id', 'INTEGER', 0, None, 0)]

In [8]:
cursor.execute('PRAGMA table_info(listings)')
cursor.fetchall()

[(0, 'id', 'INTEGER', 0, None, 0),
 (1, 'name', 'TEXT', 0, None, 0),
 (2, 'host_id', 'INTEGER', 0, None, 0),
 (3, 'location_id', 'INTEGER', 0, None, 0),
 (4, 'number_of_reviews', 'INTEGER', 0, None, 0),
 (5, 'occupancy', 'INTEGER', 0, None, 0),
 (6, 'price', 'INTEGER', 0, None, 0),
 (7, 'room_type', 'TEXT', 0, None, 0),
 (8, 'host_listings_count', 'INTEGER', 0, None, 0)]

We'll start off with some basic one table queries:

* Which listing name has the highest price?

In [28]:
cursor.execute('select name from listings order by price desc limit 1')
cursor.fetchall()

# [('Furnished room in Astoria apartment',)]

[('Furnished room in Astoria apartment',)]

* What is the id of the location with the lowest longitude?

In [29]:
cursor.execute('select id from locations order by longitude asc limit 1')
cursor.fetchall()

# [(45652,)]

[(45652,)]

* What is the greatest occupancy of a listing?

In [30]:

cursor.execute('select max(occupancy) from listings')
cursor.fetchall()
# [(365,)]

[(365,)]

* What is the average price of a listing?

In [32]:
cursor.execute('select avg(price) from listings')
cursor.fetchall()

# [(152.7206871868289,)]

[(152.7206871868289,)]

* What is the count of number of hosts?

In [33]:
cursor.execute('select count(*) from hosts')
cursor.fetchall()
# [(37457,)]

[(37457,)]

### Relationships
To help us better understand the relationships, create queries below that JOIN the tables.

### JOINs

For the following queries, use the relationships between the tables to find the solutions

* What is the longitude and latitude of the listing of the highest price?

In [34]:
cursor.execute('select longitude, latitude from locations lo JOIN listings li ON li.location_id = lo.id order by li.price desc limit 1')
cursor.fetchall()

# [(-73.91651, 40.7681)]

[(-73.91651, 40.7681)]

* What is the neighborhood id of the listing with the lowest price?

In [39]:
cursor.execute('select lo.neighborhood_id from locations lo JOIN listings li ON li.location_id = lo.id order by li.price asc limit 1')
cursor.fetchall()
# [(6,)]

[(6,)]

* What is the longitude and latitude of the listing of the lowest price?

In [37]:
cursor.execute('select longitude, latitude from locations lo JOIN listings li ON li.location_id = lo.id order by li.price asc limit 1')
cursor.fetchall()

# [(-73.95428000000001, 40.69023)]

[(-73.95428000000001, 40.69023)]

### Relations and GROUP BY

* What is the name of the host has the most number of reviews?

In [53]:
cursor.execute('''SELECT h.host_name, sum(l.number_of_reviews)
FROM listings l
JOIN hosts h ON h.id = l.host_id
GROUP By l.host_id
ORDER BY 2 desc
LIMIT 1''')
cursor.fetchall()

# [('Maya', 2273)]

[('Maya', 2273)]

* What is the name of the host with the highest average listing price?

In [54]:
cursor.execute('''SELECT h.host_name
FROM listings l
JOIN hosts h ON h.id = l.host_id
GROUP By l.host_id
ORDER BY avg(l.price) desc
LIMIT 1''')
cursor.fetchall()

# [('Jelena',)]

[('Jelena',)]

* What is the name of the host with the lowest average listing price?

In [56]:
cursor.execute('''SELECT h.host_name
FROM listings l
JOIN hosts h ON h.id = l.host_id
GROUP By l.host_id
ORDER BY avg(l.price) asc
LIMIT 1''')
cursor.fetchall()

# [('Aymeric',)]

[('Aymeric',)]

* What is the name of the neighborhood with the most number of locations

In [65]:
cursor.execute('''SELECT n.name
FROM neighborhoods n
JOIN locations l ON n.id = l.neighborhood_id
GROUP By n.name
ORDER BY count(l.id) desc
LIMIT 1''')
cursor.fetchall()


# [('Williamsburg',)]

[('Williamsburg',)]

* What are the names of the neighborhoods with 10 locations?

In [67]:
cursor.execute('''SELECT n.name
FROM neighborhoods n
JOIN locations l ON n.id = l.neighborhood_id
GROUP by n.name
HAVING count(l.id) = 10
''')
cursor.fetchall()

# [('North Riverdale',),
#  ('Great Kills',),
#  ('East Morrisania',),
#  ('Melrose',),
#  ('Bergen Beach',),
#  ('Westchester Square',)]

[('Bergen Beach',),
 ('East Morrisania',),
 ('Great Kills',),
 ('Melrose',),
 ('North Riverdale',),
 ('Westchester Square',)]

The following questions will require joins of three tables

* What is the average occupancy of each neighborhood (limit to the first five results)?

In [73]:
cursor.execute('''SELECT n.name, avg(li.occupancy)
FROM neighborhoods n
JOIN locations l ON n.id = l.neighborhood_id
JOIN listings li on l.id = li.location_id
GROUP By n.name
LIMIT 5''')
cursor.fetchall()

# [('Kensington', 281.0514285714286),
#  ('Midtown', 207.29644012944985),
#  ('Harlem', 258.4224981188864),
#  ('Clinton Hill', 269.986013986014),
#  ('East Harlem', 266.0268576544315)]

[('Allerton', 201.33333333333334),
 ('Arden Heights', 270.75),
 ('Arrochar', 109.19047619047619),
 ('Arverne', 176.57142857142858),
 ('Astoria', 255.8088888888889)]

* What is the total number of reviews for each neighborhood (limit to the first five results)?

In [78]:
cursor.execute('''SELECT n.name, sum(li.number_of_reviews)
FROM neighborhoods n
JOIN locations l ON n.id = l.neighborhood_id
JOIN listings li on l.id = li.location_id
GROUP By n.name
LIMIT 5''')
cursor.fetchall()

# [('Kensington', 2972),
#  ('Midtown', 19444),
#  ('Harlem', 75962),
#  ('Clinton Hill', 14586),
#  ('East Harlem', 36446)]

[('Allerton', 1803),
 ('Arden Heights', 31),
 ('Arrochar', 307),
 ('Arverne', 2253),
 ('Astoria', 19310)]

* Write a query that returns the name and average listing price of each neighborhood (limit to the first five results)

In [81]:
cursor.execute('''SELECT n.name, avg(li.price)
FROM neighborhoods n
JOIN locations l ON n.id = l.neighborhood_id
JOIN listings li on l.id = li.location_id
GROUP By n.name
LIMIT 5''')
cursor.fetchall()

# [('Kensington', 92.88571428571429),
#  ('Midtown', 282.7190938511327),
#  ('Harlem', 118.97404063205417),
#  ('Clinton Hill', 181.89335664335664),
#  ('East Harlem', 133.1987466427932)]

[('Allerton', 87.5952380952381),
 ('Arden Heights', 67.25),
 ('Arrochar', 115.0),
 ('Arverne', 171.7792207792208),
 ('Astoria', 117.18777777777778)]

### Conclusion
In this lab we worked on the "Has One" and "Has Many" relations in SQL. We began by mapping out the relations between the tables, which gave us a better idea of how we could then join them in our queries. We finished the lab by creating queries using JOIN clauses that connect the tables using these relationships.